# 1. Introduction

## 1.1. Project Overview

This project focuses on analyzing public sentiment from Tweets regarding Apple and Google products. The main goal is to build a Natural Language Processing (NLP) model that can classify the sentiment of a Tweet as positive, negative, or neutral based on its content.

The data, sourced from [CrowdFlower](https://data.world/crowdflower/brands-and-product-emotions), contains over 9,000 Tweets labeled by human raters. By analyzing this data, the model will help Apple and Google better understand customer perceptions of their products, allowing them to make informed decisions for marketing, customer service, and product development.

We will begin by preprocessing the Tweets, transforming them into a numerical format suitable for machine learning models, and training several classification algorithms to evaluate their performance.


## 1.2. Business Problem & Stakeholder

### Business Problem:
Public perception of tech products can heavily influence a company's sales, customer satisfaction, and brand loyalty. For companies like Apple and Google, understanding how customers feel about their products can provide valuable insights into areas for improvement, marketing strategies, and product development. 

In this project, we aim to develop a model that automatically classifies the sentiment of Tweets regarding Apple and Google products as positive, negative, or neutral. This can help companies quickly gauge public sentiment at scale, providing actionable insights for decision-making.

### Stakeholders:
- **Apple and Google Product Teams:** Use sentiment data to improve products and address customer pain points.
- **Marketing Departments:** Tailor campaigns to target sentiment-driven messaging.
- **Customer Support Teams:** Identify negative feedback more quickly to address concerns.
- **Executives/Decision Makers:** Gain a high-level view of public opinion, enabling better strategic planning.


## 1.3. Dataset Description

The dataset used in this project comes from [CrowdFlower](https://data.world/crowdflower/brands-and-product-emotions), containing over 9,000 Tweets. Each Tweet has been labeled by human raters with one of three sentiment categories: positive, negative, or neither. The dataset contains the following key columns:

### Key Features of the Dataset:
- **Text**: The actual content of the Tweet, which we will analyze for sentiment classification.
- **Brand/Product**: The specific product or brand mentioned in the Tweet, such as `iPhone`, `Google`, or `iPad`.
- **Emotion**: The sentiment label, representing whether the sentiment expressed in the Tweet is `Positive emotion`, `Negative emotion`, or neutral.

### Target Variable:
- **Emotion**: This will be the target variable, as it captures the sentiment associated with each Tweet.

The dataset will be used to train and evaluate models that can predict the sentiment of unseen Tweets based on their text content.


## 1.4. Objectives

### Specific Objective
- Develop a Natural Language Processing (NLP) model to accurately classify the sentiment of Tweets about Apple and Google products into three categories: positive, negative, and neutral.

### Additional Objectives

1. **Data Preprocessing**: Clean and prepare the Tweet data for analysis, which includes handling missing values, normalizing text (removing URLs, special characters, etc.), and tokenization.

2. **Exploratory Data Analysis (EDA)**: Conduct exploratory analysis to understand the distribution of sentiments in the dataset, identify any patterns, and visualize key aspects of the data.

3. **Model Development**: Implement and train various classification models (e.g., Logistic Regression, Support Vector Machines, and Naive Bayes) to classify the sentiment of Tweets.

4. **Model Evaluation**: Evaluate the performance of the models using appropriate metrics such as accuracy, precision, recall, and F1 score, especially focusing on multiclass classification metrics.

5. **Iterative Improvement**: Based on initial results, refine the models by incorporating advanced techniques such as feature engineering, hyperparameter tuning, or using pre-trained embeddings like Word2Vec or BERT.

6. **Conclusion and Recommendations**: Summarize the findings, discuss the model's effectiveness, and provide actionable recommendations for stakeholders based on the analysis.


# 2. Data Understanding & Exploration

## 2.1. Data Loading & Initial Exploration

In this step, we will:

1. Load the dataset.
2. Display the first few rows to understand the structure.
3. Check the data types of each column.
4. Identify missing values and duplicates.
5. Analyze the class distribution to check for potential imbalance.
6. Perform basic descriptive statistics (e.g., distribution of Tweet lengths and word counts).

In [ ]:
# Importing the necessary libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Display settings
%matplotlib inline
plt.style.use('ggplot')  # for nicer visualizations
sns.set_palette("Set2")  # Set Seaborn color palette

In [ ]:
# Load the dataset
raw_data = pd.read_csv("judge-1377884607_tweet_product_company.csv", encoding = "latin1")

In [ ]:
# Display the first 5 rows
raw_data.head()

The column names seem to be unnecessarily long, we'll rename them to improve readability.

In [ ]:
# Renaming the columns
raw_data.columns = ["Text", "Brand/Product", "Emotion"]
raw_data.head()

In [ ]:
# Checking the number of raws and columns
raw_data.shape

In [ ]:
# Getting an overview of the data types
raw_data.info()

In [ ]:
# Checking for missing values
raw_data.isnull().sum()

In [ ]:
# Checking for duplicates based on the Text column
duplicates = raw_data.duplicated(subset='Text').sum()
print(f"Number of duplicate rows based on the text column: {duplicates}")

In [ ]:
# Handle missing values in 'Text' column
raw_data['Text'] = raw_data['Text'].fillna('')  # Replace NaN with an empty string

# Descriptive statistics for tweet length and word count
raw_data['tweet_length'] = raw_data['Text'].apply(len)
raw_data['word_count'] = raw_data['Text'].apply(lambda x: len(x.split()))

print("Tweet Length Statistics:")
print(raw_data['tweet_length'].describe())

print("Word Count Statistics:")
print(raw_data['word_count'].describe())

- **Average tweet length** is *105 characters* with a max of *178 characters*.
- **Average word count** is around *18 words*, with a maximum of *33 words*.

## 2.2. Data Cleaning

In this section, we will:

- Handle missing values in the `Brand/Product` column.
- Remove any duplicate rows.
- Clean the text data by removing URLs, mentions, and special characters.

### Handling Missing Values

Since the `Brand/Product` column contains a large number of missing values, we can consider one of the following approaches:

- Dropping the column entirely if it's not necessary for the analysis.
- Imputing values, but this might be challenging as this is categorical text data.

For now, since this column is not critical to sentiment analysis, we'll drop it.

In [ ]:
# Dropping the 'Brand/Product' column due to many missing values
data_cleaned = raw_data.drop(columns=['Brand/Product'])
data_cleaned.head()

### Handling Duplicates

We'll remove the duplicate rows.

In [ ]:
# Removing duplicate rows
data_cleaned = data_cleaned.drop_duplicates()
print(f"Number of rows after removing duplicates: {data_cleaned.shape[0]}")

### Cleaning the Text Column

To prepare the Text Column for modeling, we'll:

- Remove URLs, mentions, and special characters.
- Convert text to lowercase for uniformity.

In [ ]:
import re

# Function to clean Text Column
def clean_text(text):
    text = re.sub(r"http\S+", "", text)  # Remove URLs
    text = re.sub(r"@\w+", "", text)  # Remove mentions
    text = re.sub(r"#\w+", "", text)  # Remove hashtags (optional)
    text = re.sub(r"[^a-zA-Z\s]", "", text)  # Remove special characters and numbers
    text = text.lower()  # Convert to lowercase
    text = text.strip()  # Remove leading/trailing spaces
    return text

# Applying the cleaning function to the 'Text' column
data_cleaned['cleaned_text'] = data_cleaned['Text'].apply(clean_text)
data_cleaned[['Text', 'cleaned_text']].head()

## Cleaning the Emotion Column

In [ ]:
# Checking the occurence of unique values
data_cleaned["Emotion"].value_counts()

In [ ]:
data_cleaned["Emotion"] = data_cleaned["Emotion"].replace({
    "No emotion toward brand or product": "Neutral",
    "Positive emotion": "Positive",
    "Negative emotion": "Negative"
})

data_cleaned["Emotion"].value_counts()

In [ ]:
# Setting display options to show the full text
pd.set_option('display.max_colwidth', None)  

# Displaying the tweets where Emotion is "I can't tell"
cant_tell_tweets = data_cleaned[data_cleaned["Emotion"] == "I can't tell"]["cleaned_text"]
print(cant_tell_tweets)

In [ ]:
# Drop rows where the Emotion is "I can't tell"
data_cleaned = data_cleaned[data_cleaned["Emotion"] != "I can't tell"]

On investigating the text with the cant tell emotion we decided to drop the rows because there was no clear sentiment information, as some of the texts seemed sarcastic making them less useful for accurate analysis or modeling.

## 2.3. Sentiment Class Distribution Visualization

In this step, we will visualize the sentiment class distribution to better understand the balance between different sentiment categories in the dataset. We will use a bar chart to display the count of each sentiment label: `positive`, `negative`, and `neutral`.

The bar chart will help us assess the overall distribution and check for any significant class imbalance, which may affect our modeling process later.

In [ ]:
# Setting plot size and style
plt.figure(figsize=(8, 6))

# Plotting sentiment distribution using Seaborn's countplot
sns.countplot(x='Emotion', 
              data= data_cleaned, 
              order= data_cleaned['Emotion'].value_counts().index)

# Adding title and labels
plt.title('Sentiment Distribution', fontsize=16)
plt.xlabel('Sentiment', fontsize=12)
plt.ylabel('Count', fontsize=12)

# Rotating x-axis labels for better readability
plt.xticks(rotation=45)

# Displaying the plot
plt.show()

The sentiment distribution graph clearly shows that the dataset is imbalanced, with the majority of tweets labeled as `Neutral`, followed by `Positive emotion,` while `Negative emotion` labels has significantly fewer instances.

This imbalance is something we'll need to address during the modeling phase, potentially through techniques like resampling or adjusting class weights.

## 2.4. Further Visualizations

We will now:

- **Distribution of Tweet Lengths:** A histogram to show how the lengths of the tweets are distributed.
- **Word Count Distribution:** A similar histogram for word count distribution.
- **Correlation Between Tweet Length and Sentiment:** Boxplots to visualize how the length of the tweets varies with sentiment.

### 1. Distribution of Tweet Lengths
This visualization will help us understand how the lengths of the tweets are distributed in the dataset.

In [ ]:
# Plotting distribution of tweet lengths
plt.figure(figsize=(8, 6))
sns.histplot(data_cleaned['tweet_length'], bins=30, kde=True, color='purple')
plt.title('Distribution of Tweet Lengths', fontsize=16)
plt.xlabel('Tweet Length', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.show()

The histogram above illustrates the distribution of tweet lengths within the dataset. Key observations include:

- The distribution appears to be roughly normal, with a peak around 100-125 characters.
- Most tweets fall within the range of 75 to 140 characters, suggesting that the average tweet is relatively concise, likely conforming to Twitter's character limits.
- There are fewer tweets on both extremes (very short and very long), with a noticeable decline in frequency as the tweet length approaches the maximum of 178 characters.
- The data shows some variability, but the majority of tweets are clustered around the mean, indicating a consistent tweeting style among users.

### 2. Distribution of Word Count in Tweets
We will now visualize the word count in tweets, which helps us understand how concise or detailed the tweets are.

In [ ]:
# Plotting distribution of word count
plt.figure(figsize=(8, 6))
sns.histplot(data_cleaned['word_count'], bins=30, kde=True, color='blue')
plt.title('Distribution of Word Count in Tweets', fontsize=16)
plt.xlabel('Word Count', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.show()

The histogram shows that most tweets contain between 10 and 25 words, with a peak around 20 words. Shorter tweets (around 10 words) are also common, while very short or long tweets are rare. The distribution approximates a normal curve centered at 20 words.

### 3. Correlation Between Tweet Length and Sentiment
To understand if tweet length has any relation to sentiment, we will visualize the correlation using boxplots.

In [ ]:
# Boxplot of tweet length by sentiment
plt.figure(figsize=(10, 6))
sns.boxplot(x='Emotion', y='tweet_length', data=data_cleaned)
plt.title('Tweet Length vs. Sentiment', fontsize=16)
plt.xlabel('Sentiment', fontsize=12)
plt.ylabel('Tweet Length', fontsize=12)
plt.show()

# 3. Data Preprocessing for NLP

## 3.1. Tokenization and Normalization

The goal of this step is to prepare the text for NLP tasks by transforming it into a consistent and analyzable format. We achieve this by:

1. Tokenization: Splitting the text into individual tokens (words).
2. Normalization: Converting text to lowercase and removing punctuation, numbers, and special characters.

After applying these steps, the text will be standardized, allowing for better processing in subsequent steps like vectorization.

In [ ]:
import re
import nltk
nltk.download('punkt')  # Ensure you have the required tokenizer

# Tokenization and Normalization Function
def tokenize_and_normalize(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove punctuation, numbers, and special characters
    text = re.sub(r'[^a-z\s]', '', text)
    
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    
    return tokens

# Apply the function to the cleaned text column
data_cleaned['tokens'] = data_cleaned['cleaned_text'].apply(tokenize_and_normalize)

# Display the tokenized text
data_cleaned[['cleaned_text', 'tokens']].head()

## 3.2. Stopwords Removal

In this step, we will remove common words (stopwords) that don't carry much meaningful information for text analysis. Stopwords are often words like `"the", "is", "in",` etc., which are frequently used but don’t contribute much to the overall meaning of a text.

**Goal:** Improve the quality of our text data by eliminating such words, leaving only the key terms that carry the most important meaning.

In [ ]:
from nltk.corpus import stopwords

# Load the set of English stopwords from NLTK
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Function to remove stopwords from tokenized text
def remove_stopwords(tokens):
    return [token for token in tokens if token.lower() not in stop_words]

# Apply the function to the tokens column
data_cleaned['tokens_no_stopwords'] = data_cleaned['tokens'].apply(remove_stopwords)

# Display the first few rows to check the result
data_cleaned[['tokens', 'tokens_no_stopwords']].head()

## 3.3. Stemming and Lemmatization

In this step, we will apply stemming or lemmatization to further normalize the tokens.

Stemming reduces words to their root form by removing suffixes (e.g., "running" becomes "run"). It's a more aggressive approach but might produce non-existent words.
Lemmatization is more linguistically accurate, converting words to their base or dictionary form (e.g., "running" becomes "run", and "better" becomes "good"). This method ensures the resulting words are real words.

For this project, we will use lemmatization as it tends to preserve the meaning better than stemming.

In [ ]:
# Import the necessary NLTK downloader
import nltk

# Download the 'averaged_perceptron_tagger' resource for POS tagging
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')  # Download WordNet for lemmatization
nltk.download('omw-1.4')  # Optional: Download for expanded WordNet support

# Now, proceed with the original code
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Helper function to get part of speech (POS) tags for more accurate lemmatization
def get_pos_tag(word):
    # Mapping from POS tag to first character for lemmatizer
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {
        'J': wordnet.ADJ,
        'N': wordnet.NOUN,
        'V': wordnet.VERB,
        'R': wordnet.ADV
    }
    return tag_dict.get(tag, wordnet.NOUN)

# Function to apply lemmatization to tokens
def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(token, get_pos_tag(token)) for token in tokens]

# Apply the lemmatization function to the tokens without stopwords
data_cleaned['tokens_lemmatized'] = data_cleaned['tokens_no_stopwords'].apply(lemmatize_tokens)

# Display the first few rows to check the result
data_cleaned[['tokens_no_stopwords', 'tokens_lemmatized']].head()


## 3.4. Vectorization

### Steps for Vectorization

- **Ensure Cleaned Data is Present:** We will make sure that the cleaned data (after tokenization, normalization, stopwords removal, and lemmatization) is correctly defined and contains meaningful content.

- **Perform Vectorization:** Use `CountVectorizer` for `Bag of Words` representation and `TfidfVectorizer` for `TF-IDF` representation.

In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming 'tokens_lemmatized' is a column in your data_cleaned DataFrame
# and 'Emotion' is your target variable

# Join tokens (which are lists) back into strings
data_cleaned['tokens_lemmatized_str'] = data_cleaned['tokens_lemmatized'].apply(lambda tokens: ' '.join(tokens))

X = data_cleaned['tokens_lemmatized_str']  # Features (converted to strings)
y = data_cleaned['Emotion']  # Target variable (emotion)

# Train-test split (adjust test size as needed)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize TfidfVectorizer for TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the training data, and transform the test data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Convert to DataFrame for better readability if needed (optional)
tfidf_train_df = pd.DataFrame(X_train_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names())
tfidf_test_df = pd.DataFrame(X_test_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names())

# Display TF-IDF representation for the training set
print("\nTF-IDF Representation (Training Set):")
print(tfidf_train_df.head())

### Key points about the output:

- **Sparsity:** Most of the values are 0.0, indicating that many words from the vocabulary don't appear in individual documents. This is common with text data, where only a small subset of words is present in any given document.

- **Vocabulary size:** In this case, the training set has 6,925 unique words (columns). These are the words that were identified from the training data and assigned TF-IDF scores.

- **Non-zero values:** When a word appears in a document, it is assigned a non-zero TF-IDF value. The higher the value, the more relevant that word is to the document in relation to the entire corpus.

## 3.5. Class Imbalance Handling

In [ ]:
pip install imbalanced-learn

In [ ]:
# Import necessary libraries
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Join tokens (which are lists) back into strings
data_cleaned['tokens_lemmatized_str'] = data_cleaned['tokens_lemmatized'].apply(lambda tokens: ' '.join(tokens))

X = data_cleaned['tokens_lemmatized_str']  # Features (converted to strings)
y = data_cleaned['Emotion']  # Target variable (emotion)

# Train-test split (adjust test size as needed)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize TfidfVectorizer for TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the training data, and transform the test data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Apply SMOTE to balance the classes in the training data
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_tfidf, y_train)

# Check the class distribution after applying SMOTE
print("Class distribution after SMOTE:")
print(y_train_balanced.value_counts())

# After this we can now proceed with training a model using X_train_balanced and y_train_balanced


In [ ]:
# Print shapes to confirm sizes before SMOTE
print("Before SMOTE:")
print(f"X_train_tfidf shape: {X_train_tfidf.shape}")
print(f"y_train shape: {y_train.shape}")

In [ ]:
# Print shapes after SMOTE to confirm sizes
print("After SMOTE:")
print(f"X_train_balanced shape: {X_train_balanced.shape}")
print(f"y_train_balanced shape: {y_train_balanced.shape}")

The output shows that the classes have been successfully balanced using SMOTE, with each class (Neutral, Positive, Negative) now having 4,328 samples.

# 4. Modeling

## 4.1. Model 1: Binary Classification (Positive vs. Negative)

### 4.1.1. Model Selection

We will begin with a binary classifier focusing on distinguishing between Positive and Negative emotions. For this, let's select two baseline models:

1. Naive Bayes (specifically `MultinomialNB` since we are dealing with text data).

2. Logistic Regression.

Let's start with these two models.

In [ ]:
# Import necessary libraries
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Filter out Neutral samples for binary classification in the training set
binary_train_mask = y_train_balanced != 'Neutral'
binary_test_mask = y_test != 'Neutral'

# Subset the training and test sets
X_train_binary = X_train_balanced[binary_train_mask]
y_train_binary = y_train_balanced[binary_train_mask]
X_test_binary = X_test_tfidf[binary_test_mask]
y_test_binary = y_test[binary_test_mask]

# Convert target labels to binary (1 for Positive, 0 for Negative)
y_train_binary = y_train_binary.map({'Positive': 1, 'Negative': 0})
y_test_binary = y_test_binary.map({'Positive': 1, 'Negative': 0})

# Initialize models
nb_model = MultinomialNB()
logreg_model = LogisticRegression(max_iter=1000, random_state=42)


### 4.1.2. Model Training

We will now train both models on the binary classification task.

In [ ]:
# Train Naive Bayes
nb_model.fit(X_train_binary, y_train_binary)

# Train Logistic Regression
logreg_model.fit(X_train_binary, y_train_binary)

### 4.1.3. Model Evaluation

We will evaluate the performance of both models using several metrics:

1. Accuracy
2. Precision, Recall, and F1-Score for each class.
3. ROC-AUC for binary classification.

In [ ]:
# Make predictions on the test data
y_pred_nb = nb_model.predict(X_test_binary)
y_pred_logreg = logreg_model.predict(X_test_binary)

# Evaluate Naive Bayes
print("Naive Bayes Model Evaluation:")
print("Accuracy:", accuracy_score(y_test_binary, y_pred_nb))
print("\nClassification Report (Naive Bayes):")
print(classification_report(y_test_binary, y_pred_nb))
print("\nConfusion Matrix (Naive Bayes):")
print(confusion_matrix(y_test_binary, y_pred_nb))

# Evaluate Logistic Regression
print("\nLogistic Regression Model Evaluation:")
print("Accuracy:", accuracy_score(y_test_binary, y_pred_logreg))
print("\nClassification Report (Logistic Regression):")
print(classification_report(y_test_binary, y_pred_logreg))
print("\nConfusion Matrix (Logistic Regression):")
print(confusion_matrix(y_test_binary, y_pred_logreg))

# ROC-AUC Score for Logistic Regression
y_pred_logreg_proba = logreg_model.predict_proba(X_test_binary)[:, 1]
roc_auc = roc_auc_score(y_test_binary, y_pred_logreg_proba)
print(f"ROC-AUC (Logistic Regression): {roc_auc}")

# Plot ROC Curve for Logistic Regression
fpr, tpr, _ = roc_curve(y_test_binary, y_pred_logreg_proba)
plt.figure()
plt.plot(fpr, tpr, color='blue', label=f"ROC Curve (area = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Logistic Regression')
plt.legend(loc="lower right")
plt.show()

### Model Comparison Summary

1. **Naive Bayes:**

- Accuracy: `82.7%`
- Strength in predicting `Class 1 (Positive)` with a precision of `0.92`, but struggles with `Class 0 (Negative)` with a precision of `0.52`.
- F1-score for Class 0 is relatively low `(0.58)`, reflecting issues with false positives.

2. **Logistic Regression:**

- Accuracy: `85.4%` (higher than Naive Bayes).
- Better performance overall, especially for Class 0, with improved precision (`0.58`) and F1-score (`0.63`).
- ROC-AUC: `0.87`, showing a good ability to distinguish between classes.

**Conclusion:** Logistic Regression outperforms Naive Bayes, providing better precision, recall, and overall class balance, particularly in handling the minority class (Class 0).

### Comparison of evaluation metrics

In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt
import numpy as np

# Define the metrics for both models (precision, recall, f1-score)
metrics_nb = {
    'Class 0': {'precision': 0.52, 'recall': 0.67, 'f1-score': 0.58},
    'Class 1': {'precision': 0.92, 'recall': 0.86, 'f1-score': 0.89}
}

metrics_lr = {
    'Class 0': {'precision': 0.58, 'recall': 0.68, 'f1-score': 0.63},
    'Class 1': {'precision': 0.93, 'recall': 0.89, 'f1-score': 0.91}
}

# Plot the comparison for precision, recall, and f1-score
classes = ['Class 0', 'Class 1']
metrics = ['precision', 'recall', 'f1-score']

fig, ax = plt.subplots(1, 3, figsize=(15, 5))

for i, metric in enumerate(metrics):
    nb_scores = [metrics_nb[cls][metric] for cls in classes]
    lr_scores = [metrics_lr[cls][metric] for cls in classes]

    bar_width = 0.35
    index = np.arange(len(classes))

    ax[i].bar(index, nb_scores, bar_width, label='Naive Bayes', color='blue')
    ax[i].bar(index + bar_width, lr_scores, bar_width, label='Logistic Regression', color='orange')

    ax[i].set_title(f'Comparison of {metric.capitalize()}')
    ax[i].set_xlabel('Classes')
    ax[i].set_ylabel(metric.capitalize())
    ax[i].set_xticks(index + bar_width / 2)
    ax[i].set_xticklabels(classes)
    ax[i].legend()

# Show the plot
plt.tight_layout()
plt.show()

## 4.2. Model 2: Multiclass Classification (Positive, Negative, Neutral)

In this step, we will perform multiclass classification using three models:

- Multinomial Naive Bayes
- Logistic Regression
- Random Forest

### 4.2.1. Model Selection

In [ ]:
# Import necessary libraries
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Use SMOTE balanced data (X_train_tfidf and y_train_smote)
# Multinomial Naive Bayes Classifier
nb_multiclass = MultinomialNB()
nb_multiclass.fit(X_train_tfidf, y_train_balanced)

# Logistic Regression Classifier
log_reg_multiclass = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
log_reg_multiclass.fit(X_train_tfidf, y_train_balanced)

# Random Forest Classifier
rf_multiclass = RandomForestClassifier(n_estimators=100, random_state=42)
rf_multiclass.fit(X_train_tfidf, y_train_balanced)

# Test the models on the test set
nb_pred = nb_multiclass.predict(X_test_tfidf)
log_reg_pred = log_reg_multiclass.predict(X_test_tfidf)
rf_pred = rf_multiclass.predict(X_test_tfidf)

# Evaluation for Multinomial Naive Bayes
print("Multinomial Naive Bayes Evaluation:")
print("Accuracy:", accuracy_score(y_test, nb_pred))
print(classification_report(y_test, nb_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, nb_pred))

# Evaluation for Logistic Regression
print("\nLogistic Regression Evaluation:")
print("Accuracy:", accuracy_score(y_test, log_reg_pred))
print(classification_report(y_test, log_reg_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, log_reg_pred))

# Evaluation for Random Forest
print("\nRandom Forest Evaluation:")
print("Accuracy:", accuracy_score(y_test, rf_pred))
print(classification_report(y_test, rf_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, rf_pred))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 1. Fit the models on the balanced training set
nb_multiclass = MultinomialNB()
nb_multiclass.fit(X_train_balanced, y_train_balanced)

log_reg_multiclass = LogisticRegression(max_iter=1000)
log_reg_multiclass.fit(X_train_balanced, y_train_balanced)

rf_multiclass = RandomForestClassifier()
rf_multiclass.fit(X_train_balanced, y_train_balanced)

# 2. Make predictions using the fitted models
nb_pred = nb_multiclass.predict(X_test_tfidf)
log_reg_pred = log_reg_multiclass.predict(X_test_tfidf)
rf_pred = rf_multiclass.predict(X_test_tfidf)

# Evaluation for Multinomial Naive Bayes
print("\nMultinomial Naive Bayes Evaluation:")
print("Accuracy:", accuracy_score(y_test, nb_pred))
print(classification_report(y_test, nb_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, nb_pred))

# Evaluation for Logistic Regression
print("\nLogistic Regression Evaluation:")
print("Accuracy:", accuracy_score(y_test, log_reg_pred))
print(classification_report(y_test, log_reg_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, log_reg_pred))

# Evaluation for Random Forest
print("\nRandom Forest Evaluation:")
print("Accuracy:", accuracy_score(y_test, rf_pred))
print(classification_report(y_test, rf_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, rf_pred))


### Overall Observations

- **Accuracy Comparison:**

    Binary models (especially Logistic Regression) outperformed multiclass models, achieving higher accuracy rates `(up to 85.44% for Logistic Regression)` compared to the highest multiclass accuracy of `67.47% (Random Forest)`.

- **Class Prediction Strength:**

    In binary classification, both models excelled at classifying Positive instances, while the multiclass models struggled with Negative class predictions across the board. Multinomial Naive Bayes, in particular, had low precision and F1-scores for the Negative class.

Given the performance discrepancies, the next step will involve hyperparameter tuning to optimize the multiclass models, particularly focusing on improving the classification of the Negative class and overall model accuracy. This will be accomplished using `GridSearchCV` for hyperparameter optimization on the selected multiclass models.

### 4.2.2. Hyperparameter Tuning

We'll use `GridSearchCV` to optimize hyperparameters for the multiclass models, focusing on the following models: *Multinomial Naive Bayes*, *Logistic Regression*, and *Random Forest*.

We'll define a parameter grid for each model and use GridSearchCV to find the best parameters based on `cross-validated performance`.

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Define parameter grids for each model
param_grid_nb = {
    'alpha': [0.1, 0.5, 1.0, 1.5, 2.0]  # Smoothing parameter for MultinomialNB
}

param_grid_lr = {
    'C': [0.01, 0.1, 1, 10, 100],  # Inverse of regularization strength
    'max_iter': [100, 200, 300]  # Maximum number of iterations for convergence
}

param_grid_rf = {
    'n_estimators': [50, 100, 200],  # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the trees
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split an internal node
}

# Initialize models
nb_model = MultinomialNB()
lr_model = LogisticRegression()
rf_model = RandomForestClassifier()

# Initialize GridSearchCV for each model
grid_nb = GridSearchCV(nb_model, param_grid_nb, cv=5, scoring='accuracy', verbose=1)
grid_lr = GridSearchCV(lr_model, param_grid_lr, cv=5, scoring='accuracy', verbose=1)
grid_rf = GridSearchCV(rf_model, param_grid_rf, cv=5, scoring='accuracy', verbose=1)

# Fit the models on the balanced training data
print("Tuning Multinomial Naive Bayes...")
grid_nb.fit(X_train_balanced, y_train_balanced)

print("Tuning Logistic Regression...")
grid_lr.fit(X_train_balanced, y_train_balanced)

print("Tuning Random Forest...")
grid_rf.fit(X_train_balanced, y_train_balanced)

# Best parameters for each model
print("\nBest parameters for Multinomial Naive Bayes:", grid_nb.best_params_)
print("Best parameters for Logistic Regression:", grid_lr.best_params_)
print("Best parameters for Random Forest:", grid_rf.best_params_)


### 4.2.3. Model Evaluation

## 4.3. Advanced Models

### 4.3.1. Experiment with Word2Vec or GloVe embeddings

### 4.3.2. Use BERT for Sentiment Classification

### 4.3.3. Model Evaluation: (comparison with the simpler models above)

# 5. Model Explainability

## 5.1. SHAP or LIME Interpretation

Use SHAP or LIME to explain which features (words or n-grams) contributed most to model decisions.

- *Visualize and interpret feature importance for each sentiment class.*

# 6. Validation Strategy

## 6.1. Cross-Validation

## 6.2. Final Model Selection

# 7. Conclusion

## 7.1. Business Insights

## 7.2. Model Performance Overview

## 7.3. Future Work